In [1]:
# Dependencies to pull API
import requests
import json
# Google developer API key
from config import gkey

In [2]:
# Import Dependencies for Database
from config import password
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import psycopg2
import pandas as pd
import time

In [3]:
location_wine_df = pd.read_csv("./Data/US_wine_data.csv")

In [ ]:
location_wine_df.columns

In [4]:
region_df = location_wine_df.drop(columns = ['description', 'points', 'price', 'province',
       'region_2', 'title', 'variety', 'winery','price_bins'], axis = 1)
region_df = region_df.drop_duplicates()

In [5]:
region_list = list(region_df['region_1'])
region_list

['Willamette Valley',
 'Lake Michigan Shore',
 'Napa Valley',
 'Alexander Valley',
 'Central Coast',
 'Virginia',
 'Oregon',
 'Paso Robles',
 'Sonoma Coast',
 'Clarksburg',
 'Dry Creek Valley',
 'Sonoma Valley',
 'McMinnville',
 'Lake County',
 'Monticello',
 'North Coast',
 'Columbia Valley (WA)',
 'Santa Ynez Valley',
 'California',
 'Howell Mountain',
 'Calistoga',
 'Eola-Amity Hills',
 'Ancient Lakes',
 'Knights Valley',
 'Sonoma County',
 'Finger Lakes',
 'Santa Clara Valley',
 'Yountville',
 'Rockpile',
 'Santa Lucia Highlands',
 'Spring Mountain District',
 'Monterey',
 'Carmel Valley',
 'Santa Cruz Mountains',
 'Edna Valley',
 'Russian River Valley',
 'Arroyo Seco',
 'Anderson Valley',
 'Nevada County',
 'Cayuga Lake',
 'Santa Barbara County',
 'Diamond Mountain District',
 'Mendocino',
 'Sonoma Mountain',
 'Amador County',
 'St. Helena',
 'Cole Ranch',
 'Santa Maria Valley',
 'Dundee Hills',
 'Arroyo Grande Valley',
 'Mendocino County',
 'Rutherford',
 'Walla Walla Valley (WA)

In [13]:
# Target valley
#region = "Sonoma Valley"

# Build the endpoint URL
valley_url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?inputtype=textquery&key='+ gkey



In [14]:
regions = str(region_df)
regions.format

<function str.format>

In [ ]:
# Run a request to endpoint and convert result to json
geo_data = requests.get(valley_url).json()

# Print the json
print(geo_data)

In [15]:
# Create an empty list to hold the weather data.
place_id_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [16]:
# Loop through all the regions in the list.
for i, region in enumerate(region_list):

 # Group regions in sets of 25 for logging purposes.
    if (i % 25 == 0 and i >= 25):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each region_id.
    placeid_url = valley_url + '&input='+ region
    

    # Add a one second interval between queries to stay within API query limits
    time.sleep(1)
    
    # Log the URL, record, and set numbers and the region.
    print(f"Processing Record {record_count} of Set {set_count} | {region}")
    # Add 1 to the record count.
    record_count += 1 
    
# Run an API request for each of the regions.
    try:
        # Parse the JSON and retrieve data.
        region = requests.get(placeid_url).json()
        # Parse out the needed data.
        place_id = region['candidates'][0]['place_id']
        # Append the region information into place_id_data list.
        place_id_data.append({"region": region.title(), 
                              "Place_id": place_id})

# If an error is experienced, skip the region.
    except:
        print("region not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | Willamette Valley
region not found. Skipping...
Processing Record 2 of Set 1 | Lake Michigan Shore
region not found. Skipping...
Processing Record 3 of Set 1 | Napa Valley
region not found. Skipping...
Processing Record 4 of Set 1 | Alexander Valley
region not found. Skipping...
Processing Record 5 of Set 1 | Central Coast
region not found. Skipping...
Processing Record 6 of Set 1 | Virginia
region not found. Skipping...
Processing Record 7 of Set 1 | Oregon
region not found. Skipping...
Processing Record 8 of Set 1 | Paso Robles
region not found. Skipping...
Processing Record 9 of Set 1 | Sonoma Coast
region not found. Skipping...
Processing Record 10 of Set 1 | Clarksburg
region not found. Skipping...
Processing Record 11 of Set 1 | Dry Creek Valley
region not found. Skipping...
Processing Record 12 of Set 1 | Sonoma Valley
region not found. Skipping...
Processing Record 13 of Set 1 | McMinnville
region not found. Skipping...
Processing Record 14 of Set

region not found. Skipping...
Processing Record 8 of Set 5 | Old Mission Peninsula
region not found. Skipping...
Processing Record 9 of Set 5 | Dunnigan Hills
region not found. Skipping...
Processing Record 10 of Set 5 | Coombsville
region not found. Skipping...
Processing Record 11 of Set 5 | Oakville
region not found. Skipping...
Processing Record 12 of Set 5 | America
region not found. Skipping...
Processing Record 13 of Set 5 | Middleburg
region not found. Skipping...
Processing Record 14 of Set 5 | Applegate Valley
region not found. Skipping...
Processing Record 15 of Set 5 | San Benito County
region not found. Skipping...
Processing Record 16 of Set 5 | San Luis Obispo County
region not found. Skipping...
Processing Record 17 of Set 5 | Leelanau Peninsula
region not found. Skipping...
Processing Record 18 of Set 5 | Outer Coastal Plain
region not found. Skipping...
Processing Record 19 of Set 5 | Sonoma-Napa
region not found. Skipping...
Processing Record 20 of Set 5 | Santa Barb

KeyboardInterrupt: 

In [ ]:
placeid_df = pd.DataFrame(place_id_data)
placeid_df.head(10)

In [ ]:
place_id = geo_data['candidates'][0]['place_id']
place_id

In [ ]:
place_url = ('https://maps.googleapis.com/maps/api/place/details/json?''place_id=ChIJd517gUOshYARReDmDiHtgAM&key={0}').format(gkey)

In [ ]:
place_data = requests.get(place_url).json()
print(place_data)

In [ ]:
latitude = place_data['result']['geometry']['location']['lat']
latitude


In [ ]:
longitude = place_data['result']['geometry']['location']['lng']
longitude